In [ ]:
%matplotlib inline
# To install bioscrape for the simulations, follow these instructions: https://github.com/ananswam/bioscrape/wiki/Installation

import numpy as np
import pylab as plt
from bioscrape.lineage import (
    LineageCSimInterface,
    LineageModel,
    LineageSSASimulator,
    LineageVolumeCellState,
)
from bioscrape.types import Model


def photobleach(mag, events):
    eqn = "_pb=" + str(mag) + "*("
    flag = False
    for e in events:
        if flag:
            eqn += " + "
        eqn += "Heaviside(t-" + str(e[0]) + ") - Heaviside(t-" + str(e[0] + e[1]) + ")"
        flag = True
    return eqn + ")"


ssa_simulator = LineageSSASimulator()
timepoints = np.linspace(0, 1000, 1000)
Tpb = 1
# pb_eqn = photobleach(.05,[(0,Tpb), (10,Tpb), (20,Tpb), (30,Tpb), (40,Tpb), (50,Tpb), (60,40)])
pb_eqn = photobleach(0.008, [(0, 600)])

k = 0
g = 0.1
N = 2000
F = 200
rxn = [
    (["GR"], ["G"], "massaction", {"k": "pb"}),
    (["GR"], ["R"], "massaction", {"k": "pb"}),
    (["R"], [], "massaction", {"k": "pb"}),
    (["G"], [], "massaction", {"k": "pb"}),
    ([], ["GR"], "massaction", {"k": k}),
]


print("Simulating Model")
plt.figure(figsize=(10, 10))
plt.title("Average Initial count = " + str(F) + ", Fluorescence gain = 10", fontsize=20)
plt.xlabel("Time", fontsize=20)
plt.ylabel("N molecules", fontsize=20)
Xsamp1 = []
Xsamp2 = []
Xsamp3 = []
Gsamp = np.zeros((1000, N))
Rsamp = np.zeros((1000, N))
pb_noise = np.random.normal(0, 0.0009, N)
print(timepoints[150])
for i in range(N):
    GR0 = F  # np.random.poisson(F)
    #     print(GR0)
    #     pb_noise = np.random.normal(0,.0005)
    pb = 0.003 + pb_noise[i]
    LM = LineageModel(
        reactions=rxn,
        initialize_model=False,
        initial_condition_dict={"GR": GR0, "G": 0, "R": 0},
        parameters=[("pb", pb)],
    )
    #     LM.create_rule(rule_type="assignment",rule_attributes={'equation':pb_eqn}, rule_frequency="repeated")
    LM.py_initialize()
    stoch_result = ssa_simulator.py_SimulateSingleCell(timepoints, Model=LM)
    stoch_sim_output = stoch_result.py_get_result()

    GR_ind = LM.get_species_index("GR")
    G_ind = LM.get_species_index("G")
    R_ind = LM.get_species_index("R")
    #     nu = 10 + np.random.normal(0,.0001,stoch_sim_output[:, GR_ind].shape)
    nu = 10  # + np.random.normal(0,.001)

    GR = nu * stoch_sim_output[:, GR_ind]
    G = nu * stoch_sim_output[:, G_ind]
    R = nu * stoch_sim_output[:, R_ind]
    #     print(X.shape)

    Gsamp[:, i] = (G + GR) * np.exp(
        0 * np.random.normal(pb_noise[i], np.abs(pb_noise).mean() / 4) * timepoints
    )  # + np.random.normal(300,5,G.T.shape)
    Gsamp[:, i] = Gsamp[:, i]  # - np.nanmin(Gsamp[:,i])
    Rsamp[:, i] = R + GR
    plt.plot(timepoints, Gsamp[:, i])
# plt.ylim([0,300])
plt.xlim([0, 1000])
# plt.figure()
# plt.hist(Xsamp,bins=10)
# print(np.array(Xsamp).shape)
# mu = np.mean(Xsamp)
# p = mu/GR0
# print(mu, np.var(Xsamp)/(p*(1-p)),np.var(Xsamp),p, p**2)
# print(Xsamp1,Xsamp2)
# plt.plot(timepoints, volume, label = "Volume")
# plt.legend()

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pylab as plt
from astropy.stats.biweight import biweight_midvariance
from matplotlib import cm
from scipy.integrate import simps, trapz
from scipy.optimize import curve_fit


def func(x, a, b):
    return a * np.exp(-b * x)


def filter_df(df, prop_dict):
    processed_df = df.copy()
    for prop in prop_dict:
        processed_df = processed_df[
            (processed_df[prop] > prop_dict[prop][0])
            & (processed_df[prop] < prop_dict[prop][1])
        ]

    return processed_df


def tau_interp(y, t, kbar):
    k = curve_fit(func, t, y)[0][1]
    #     k = np.polyfit(t[0:600],np.log(y[0:600]),1)[0]
    #     k = np.mean((y[:-1] - y[1:])/(y[:-1]))
    #     print(k,kbar)
    tau = (kbar / k * t).astype(np.int)
    tau = tau[tau < y.shape[0]]
    y_tau = y[tau]
    if y.shape[0] > tau.shape[0]:
        y_tau = np.pad(
            y_tau, (0, t.shape[0] - tau.shape[0]), "constant", constant_values=np.nan
        )
    #     print(type(y_tau))
    return pd.Series(y_tau)


def get_stats(df):
    trajectories = df
    #     x = np.arange(0,1000)
    #     k = df.apply(lambda f: np.polyfit(x[0:600],np.log(f[0:600]),1)[0],axis=1)
    k = df.apply(lambda f: curve_fit(func, timepoints, f)[0][1], axis=1)
    #     k = df.apply(lambda f: np.nanmean((f.values[:-1] - f.values[1:])/(f.values[:-1])))
    #     k_n = k.values - k.mean()
    #     k_n = np.random.normal(pb_noise,np.abs(pb_noise).mean()/4, pb_noise.shape)
    #     k_n = pb_noise
    #     y = trajectories.multiply(np.exp(k_n[:,np.newaxis]*timepoints[np.newaxis,:]))

    y = df.apply(lambda f: tau_interp(f.values, timepoints, np.nanmean(k)), axis=1)
    print(y.shape)
    p = y.apply(lambda x: x / x[0], axis=1)
    pbar = p.mean()
    mu = y.mean()
    sigma2 = (p - pbar) ** 2

    return y, pbar, mu, sigma2


def nu_int(pbar, mu, sigma, q=1):
    nu_dict = {}  # pd.Series()
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    y = sigma2
    #     print(y)
    dp = pbar.values
    dp = dp[pbar.values > 1 - q]
    #     print(y.shape, )
    f = y.iloc[pbar.values > 1 - q]

    nu_dict[name] = cq * simps(f, dp)

    return pd.Series(nu_dict)


def fluct_plot(pbar, mu, sigma2, q=1):
    #     print(hist_df)
    #     print(f'Total number of cells is {np.sum(hist_df.values)}')

    plt.figure(figsize=(12, 8))
    cmap = cm.get_cmap("coolwarm")

    y = sigma2 * mu[0]
    y[np.isnan(y)] = 0

    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    #     cq = -1/(q*(1/2 - (2/3)*q**2))

    nu = np.nanmean(
        cq * simps(y.T.iloc[pbar.values > 1 - q].T, pbar.iloc[pbar.values > 1 - q])
    )
    #     qind = (pbar.values > (1/2 - q)) & (pbar.values < (1/2 + q))
    #     print(cq,pbar.iloc[qind],y.T.iloc[qind].T)
    #     nu = np.nanmean(cq*simps(y.T.iloc[qind].T,pbar.iloc[qind]))

    #     c = (name.left - imin)/(imax - imin)
    plt.scatter(1 - pbar, np.nanmean(y, axis=0), color="k", label="Single-cell data")
    #     plt.fill_between(1-pbar,np.nanmean(y,axis=0),alpha=.1,color='r')

    p = np.linspace(0, 1, 1000)
    plt.plot(
        p,
        nu * p * (1 - p),
        linewidth=6,
        alpha=0.5,
        color="g",
        label="Theoretical curve",
    )
    p_left = np.linspace(0, q, 100)
    p_right = np.linspace(q, 1, 100)

    plt.fill_between(
        p_left,
        nu * p_left * (1 - p_left),
        alpha=0.3,
        color="c",
        label="Integrated area",
    )
    plt.fill_between(
        p_right,
        nu * p_right * (1 - p_right),
        alpha=0.3,
        color="r",
        hatch="/",
        label="Interpolated area",
    )

    plt.vlines(
        q, 0, nu * q * (1 - q), alpha=0.5, linewidth=5, color="m", label=f"q = {q}"
    )
    plt.title(
        rf"$\nu =  {-cq:.1f} \cdot \int\frac{{\hat{{\sigma}}^2}}{{f_{{max}}}}dp$ = {nu:.6f}",
        fontsize=20,
        pad=20,
    )
    plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
    plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
    plt.legend()


#     plt.ylim(bottom=0,top=.0004)

In [ ]:
df = pd.DataFrame(Gsamp.T)

y, pbar, mu, sigma2 = get_stats(df)
plt.plot(y.T)
q = 1 / 4
# nu_df = nu_int(pbar,mu,sigma2,q)
# print(nu_df)

fluct_plot(pbar, mu, sigma2, q)

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pylab as plt

# G = 10

df = pd.DataFrame(Gsamp.T)  # + np.random.normal(0,1,Gsamp.T.shape))
bins = np.arange(df[0].min() - 1, df[0].max() + 1, 100)
df["bin"] = pd.cut(df[0], bins=bins)
p = df.apply(lambda x: x[:-1:] / x[0], axis=1)
p["bin"] = df["bin"]

tau = 30
pbar = p.groupby(p["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
mu = df.groupby(df["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
sigma2 = df.groupby(df["bin"]).var(ddof=0)[df.groupby(df["bin"]).size() > tau]

y = sigma2.div(mu[0].values, axis="rows")
x = pbar * (1 - pbar)
r = y.values.flatten() / x.values.flatten()

linfit = np.polyfit(x.values.flatten(), y.values.flatten(), 1)
linfit_fn = np.poly1d(linfit)

plt.figure(figsize=(12, 8))
plt.scatter(x, y)

xfit = np.linspace(0, 0.25, 100)
nu_bar = np.round(np.mean(r[np.isfinite(r)]), 2)
plt.plot(xfit, nu_bar * xfit, "-r", linewidth=6)

plt.title(
    r"$\nu = \frac{\hat{\sigma}^2}{f_{max}\hat{p}(1-\hat{p})}$ =" + str(nu_bar),
    fontsize=20,
    pad=20,
)
plt.xlabel(r"$\hat{p}(1-\hat{p})$", fontsize=20)
plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
bin_hist = df.groupby(df["bin"]).size()
print(bin_hist[bin_hist > tau])
print(r"mean ratio: ", np.mean(r[np.isfinite(r)]))
print("median ratio: ", np.median(r[np.isfinite(r)]))

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pylab as plt
from matplotlib import cm
from scipy.integrate import simps

# G = 10
df = pd.DataFrame(Gsamp.T)  # + np.random.normal(0,1,Gsamp.T.shape))
# df = 1000 +df[df[0] > 1000]
bins = np.arange(df[0].min() - 1, df[0].max() + 1, 50)
df["bin"] = pd.cut(df[0], bins=bins)
p = df.apply(lambda x: x[:-1:] / x[0], axis=1)
p["bin"] = df["bin"]

tau = 30
pbar = p.groupby(p["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
mu = df.groupby(df["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
sigma2 = df.groupby(df["bin"]).var(ddof=0)[df.groupby(df["bin"]).size() > tau]

y = sigma2.div(mu[0].values, axis="rows")
x = pbar
r = y.values.flatten() / x.values.flatten()

linfit = np.polyfit(x.values.flatten(), y.values.flatten(), 1)
linfit_fn = np.poly1d(linfit)

plt.figure(figsize=(12, 8))
# plt.scatter(x,y)

xfit = np.linspace(0, 0.25, 100)
nu_bar = np.round(np.mean(r[np.isfinite(r)]), 2)
# plt.plot(xfit,nu_bar*xfit,'-r',linewidth=6)

plt.title(
    r"$\nu = \frac{\hat{\sigma}^2}{f_{max}\hat{p}(1-\hat{p})}$ =" + str(nu_bar),
    fontsize=20,
    pad=20,
)
plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
bin_hist = df.groupby(df["bin"]).size()
print(bin_hist[bin_hist.values > tau])
print(r"mean ratio: ", np.mean(r[np.isfinite(r)]))
print("median ratio: ", np.median(r[np.isfinite(r)]))

nu_int = []

cmap = cm.get_cmap("coolwarm")
imax = x.index[-1].left
imin = x.index[0].left
for name, group in x.groupby("bin"):
    nu_int.append(-6 * simps(y.loc[name].values, x.loc[name].values))
    dp = x.loc[name].values
    dp = dp[x.loc[name].values < 1 / 2]
    f = y.loc[name].values[x.loc[name].values < 1 / 2]
    #     nu_int.append(-12*simps(f,dp))
    c = (name.left - imin) / (imax - imin)

    plt.scatter(1 - x.loc[name].values, y.loc[name].values, color=cmap(c), label=name)
    plt.legend()
print(nu_int, np.mean(nu_int))
plt.title(
    r"$\nu = 6\int\frac{\hat{\sigma}^2}{f_{max}}dp$ =" + str(np.mean(nu_int)),
    fontsize=20,
    pad=20,
)

In [ ]:
import numpy as np

%matplotlib inline
import pylab as plt

p = np.arange(0, 0.5, 0.001)
# print(p)
g = 10
N = 1000
X = g * np.random.binomial(N, p, (10, p.size))
v = np.var(X, axis=0)
# print(v)
plt.figure(figsize=(12, 8))

plt.scatter(p * (1 - p), v / (N * g))
m, b = np.polyfit(p * (1 - p), v / (N * g), 1)
print(m, b)

plt.plot(p, m * p + b, "-r", linewidth=6)
plt.xlim([0, 0.25])

In [ ]:
def photobleach(mag, events):
    eqn = "_pb=" + str(mag) + "*("
    flag = False
    for e in events:
        if flag:
            eqn += " + "
        eqn += "Heaviside(t-" + str(e[0]) + ") - Heaviside(t-" + str(e[0] + e[1]) + ")"
        flag = True
    return eqn + ")"


photobleach(0.2, [(1, 2), (5, 3)])


def fp_label(fp):
    s = ""
    for c in fp:
        s += fp[c] * c
    return s


def rxn_tree(base_fp):
    rxn_list = []
    fp_list = [base_fp]
    for fp in fp_list:
        for c in fp:
            if fp[c] > 0:
                fp_child = fp.copy()
                fp_child[c] -= 1
                #                 print(fp_child[c],fp, fp_child)
                rxn = ([fp_label(fp)], [fp_label(fp_child)], "massaction", {"k": "pb"})
                if rxn not in rxn_list:
                    rxn_list.append(rxn)
                fp_list.append(fp_child)
    return rxn_list


rxn_tree({"R": 2, "G": 1, "Y": 3})

In [ ]:
%matplotlib inline
import numpy as np
import pylab as plt
import pystan

pb_model = """
data {
    int<lower=0> J;
    int<lower=0> E;
    int<lower=0> C;
    real<lower=0> M;
    int f[E, J, C]; 
}
parameters {
    real<lower=0, upper = 1> p[E];
    real<lower=M>  N;
    //real<lower=.05,upper=.15> G;
}
model {   
    for (e in 1:E) {
        for (j in 1:J) {
            for(c in 1:C) {
               target += lchoose(N,f[e,j,c]) + lmultiply(f[e,j,c], p[e]) + lmultiply(N-f[e,j,c], 1-p[e]);
                }
            }
        }
}
"""
sm = pystan.StanModel(model_code=pb_model)

In [ ]:
import arviz as az

# ydata =  np.array([[int(k) for k in Xsamp1],[int(k) for k in Xsamp2]])
# ydata =  np.array([[int(k) for k in Xsamp1]])
# ydata =  np.array([[int(k) for k in Xsamp1],[int(k) for k in Xsamp2],[int(k) for k in Xsamp3]])
ydata = np.stack([Gsamp, Rsamp], axis=2).astype("int")
fdata = ydata
print(fdata.max())
pb_data = {
    "J": fdata.shape[1],
    "E": fdata.shape[0],
    "C": fdata.shape[2],
    "M": fdata.max(),
    "f": fdata,
}

fit = sm.sampling(data=pb_data, iter=1000, chains=4, control={"adapt_delta": 0.99})
inference_data = az.from_pystan(posterior=fit)  # posterior kw is optional
az.plot_trace(inference_data)

print(fit)

In [ ]:
%matplotlib inline
import numpy as np
import pylab as plt
import pystan

pb_model = """
data {
    int<lower=0> J;
    int<lower=0> E;
    int<lower=0> C;
    real<lower=0> M;
    real p[E, J, C]; 
}
parameters {
    real<lower=0>  N;
    real<lower=0, upper=1> pbar[E,C];
    real<lower=0> sigma2[E,C];
    real<lower=0> s[E,C];
}
model {   
    for (e in 1:E) {
            for(c in 1:C) {
                p[e,:,c] ~ normal(pbar[e,c], sqrt(pbar[e,c]*(1-pbar[e,c])/J));
                sigma2[e,c] ~ normal(N*pbar[e,c]*(1-pbar[e,c]),s[e,c]);
                }
        }
}
"""
sm = pystan.StanModel(model_code=pb_model)

In [ ]:
pdata = np.stack([Gsamp, Rsamp], axis=2)
print(ydata.shape[1])
pb_data = {
    "J": pdata.shape[1],
    "E": pdata.shape[0],
    "C": pdata.shape[2],
    "M": pdata.max(),
    "p": pdata,
}

fit = sm.sampling(data=pb_data, iter=2000, chains=4, control={"adapt_delta": 0.99})
plt.tight_layout(w_pad=4, h_pad=6)

fit.plot()
print(fit)

##### 